# Introduction to cudnn frontend python API
This notebook is an introduction to cudnn FE graph python API and how to perform a single fprop convolution

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/NVIDIA/cudnn-frontend/blob/main/samples/python/00_introduction.ipynb)

## Prerequisites for running on Colab
This notebook requires an NVIDIA GPU. If `nvidia-smi` fails, go to Runtime -> Change runtime type -> Hardware accelerator and confirm a GPU is selected.

In [3]:
# get_ipython().system('nvidia-smi')

If running on Colab, you will need to install the cudnn python interface.

In [4]:
# get_ipython().system('pip install nvidia-cudnn-cu12')
# get_ipython().system('pip install nvidia-cudnn-frontend')
# get_ipython().system('pip3 install --pre torch --index-url https://download.pytorch.org/whl/nightly/cu128')

#### General Setup

In [5]:
import cudnn

In [6]:
print(cudnn.backend_version())

handle = cudnn.create_handle()

91002


`backend_version()` prints the cudnn backend version. Eg. 90000

`handle` is a pointer to an opaque structure holding the cuDNN library context. The cuDNN library context must be created using `create_handle()` and the returned handle must be passed to all subsequent library function calls as needed. The context should be destroyed at the end using `destroy_handle()`. 

The context is associated with only one GPU device, the current device at the time of the call to `create_handle()`. However, multiple contexts can be created on the same GPU device.

In [7]:
graph = cudnn.pygraph(
    handle=handle,
    name="cudnn_graph_0",
    io_data_type=cudnn.data_type.HALF,
    compute_data_type=cudnn.data_type.FLOAT,
)

`pygraph` is the subgraph that is provided to the cudnn for execution.

Each component in the graph has an optional `name` for future reference. (Optional)

The `io_data_type` provides the data type of the input and output tensors of the graph. This can be overridden by actual tensor data type. (Optional)

The `compute_data_type` provides the data type in which computation will happen. This can be overridden by actual compute data type of the individual operation. (Optional)

In [8]:
X = graph.tensor(
    name="X",
    dim=[8, 64, 56, 56],
    stride=[56 * 56 * 64, 1, 56 * 64, 64],
    data_type=cudnn.data_type.HALF,
)

In [9]:
W = graph.tensor(name="W", dim=[32, 64, 3, 3], stride=[3 * 3 * 64, 1, 3 * 64, 64])

`graph.tensor` creates an entry edge to the graph. The main attributes of the tensor class are `dim`, `stride` and `data_type`. 
Some other attributes are `is_virtual` (mainly used for interior nodes in graph), `is_pass_by_value` for scalar tensors.

Note that the "W" tensor above did not have data_type. Its data type is deduced from the `pygraph.io_data_type` that was specified above.

In [10]:
Y = graph.conv_fprop(
    X,
    W,
    padding=[1, 1],
    stride=[1, 1],
    dilation=[1, 1],
    compute_data_type=cudnn.data_type.FLOAT,
)

Perform a `convolution forward` operation with padding as [1,1] on the input X tensor.

Other parameters are `compute_data_type`, `stride`, `dilation`. See `help (cudnn.pygraph.conv_fprop)`

In [11]:
Y.set_output(True)

[{"data_type":null,"dim":[],"is_pass_by_value":false,"is_virtual":false,"name":"0::Y","pass_by_value":null,"reordering_type":"NONE","stride":[],"uid":0,"uid_assigned":false}]

By default the output of any operation is virtual (does not have device pointer associated). This is because the output can be fed as input to the next operation in graph. In order to terminate the graph, or to mark the tensor non-virtual we need to set the output. 

In [12]:
graph.build([cudnn.heur_mode.A])
# print(graph)

Following things happen in the above call
- validation of inputs, outputs and output shape deduction.
- Lowering pass into the cudnn dialect.
- Heuristics query to determine which execution plan to run.
- Runtime compilation of the plan if needed

In following notebooks, we will see that this function gets split into its constituents to have a better control over each phase.

Use the `print` function to inspect the graph after the shape and datatype deduction.

In [ ]:
import torch

X_gpu = torch.randn(
    8, 64, 56, 56, requires_grad=False, device="cuda", dtype=torch.float16
).to(memory_format=torch.channels_last)
W_gpu = torch.randn(
    32, 64, 3, 3, requires_grad=False, device="cuda", dtype=torch.float16
).to(memory_format=torch.channels_last)
Y_gpu = torch.zeros(
    8, 32, 3, 3, requires_grad=False, device="cuda", dtype=torch.float16
).to(memory_format=torch.channels_last)
workspace = torch.empty(graph.get_workspace_size(), device="cuda", dtype=torch.uint8)

/home/ec2-user/gpu/.venv/lib64/python3.12/site-packages/torch/_subclasses/functional_tensor.py:279: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /pytorch/torch/csrc/utils/tensor_numpy.cpp:84.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


In [16]:
print(graph)

{
    "context": {
        "compute_data_type": "FLOAT",
        "intermediate_data_type": null,
        "io_data_type": "HALF",
        "name": "",
        "sm_count": -1
    },
    "cudnn_backend_version": "9.10.2",
    "cudnn_frontend_version": 11401,
    "json_version": "1.0",
    "nodes": [
        {
            "compute_data_type": "FLOAT",
            "dilation": [1,1],
            "inputs": {
                "W": "W",
                "X": "X"
            },
            "math_mode": "CROSS_CORRELATION",
            "name": "0",
            "outputs": {
                "Y": "0::Y"
            },
            "post_padding": [1,1],
            "pre_padding": [1,1],
            "stride": [1,1],
            "tag": "CONV_FPROP"
        }
    ],
    "tensors": {
        "0::Y": {
            "data_type": "HALF",
            "dim": [8,32,56,56],
            "is_pass_by_value": false,
            "is_virtual": false,
            "name": "0::Y",
            "pass_by_value": null,
        

Here we are using torch to create GPU tensor. Note that, cudnn FE supports any DLPack interface.

In [14]:
graph.execute({X: X_gpu, W: W_gpu, Y: Y_gpu}, workspace, handle=handle)

The execute call launches the kernel for execution on the GPU device.

In [15]:
print(Y_gpu)

tensor([[[[-2.5137e+00,  5.3156e+01, -2.0797e+01],
          [ 2.4281e+01, -3.2938e+01,  1.2383e+01],
          [ 9.1719e+00, -5.7031e-01, -1.0609e+01]],

         [[ 3.3594e+01,  2.6922e+01,  1.8188e+01],
          [-3.8688e+01, -7.4062e+00,  3.5156e+00],
          [-1.4711e+01, -1.4836e+01, -2.8859e+01]],

         [[ 2.4812e+01, -1.4016e+01, -1.6891e+01],
          [ 1.0455e-01,  4.5195e+00,  2.5719e+01],
          [-1.3945e+01,  1.9484e+01, -1.6953e+01]],

         ...,

         [[-3.5312e+01, -2.9531e+01,  1.0062e+01],
          [ 1.1203e+01,  1.0633e+01, -9.7500e+00],
          [ 2.0516e+01, -7.7148e-02,  2.1688e+01]],

         [[-1.5586e+01,  4.4750e+01,  2.0812e+01],
          [ 2.7578e+00, -6.8398e+00, -2.9812e+01],
          [ 3.7375e+01,  2.9473e+00,  7.0820e+00]],

         [[-1.0953e+01, -2.0688e+01,  2.6594e+01],
          [ 6.9678e-01,  1.2367e+01,  4.5703e+00],
          [-7.0781e+00, -2.4344e+01,  1.4719e+01]]],


        [[[-3.0969e+01,  2.0359e+01,  1.7766e+01],
  